# Setup

In [65]:
import pandas as pd
from pathlib import Path

In [66]:
#input args
log_name = 'logs/LOG_166.csv'
ticks_per_second_mb = 1000
out_dir = './out'

#derived args
split_logs_dir = f'{out_dir}/split_logs/'

In [67]:
#create output folders
Path(f'{out_dir}/split_logs').mkdir(parents=True, exist_ok=True)
Path(f'{out_dir}/plots').mkdir(parents=True, exist_ok=True)

# Full Log

In [68]:
full_log = pd.read_csv(log_name, delimiter = ';')
full_log.index.rename('Log Entry', inplace = True)
full_log.head()

,Timestamp,Log Entry Type,Data
Log Entry,,,
0,201,1,"1,3,0,0,0,0,0,0,0,0,0,0"
1,201,1,"2,3,0,0,0,0,0,0,0,0,0,0"
2,203,1,"3,3,0,0,0,0,0,0,0,0,0,0"
3,511,1,"1,1,0,0,0"
4,511,1,"1,2,0,0,0,0,0,0,0"


In [69]:
#convert timestamp ticks to seconds
full_log['Time_s'] = full_log['Timestamp']/ticks_per_second_mb

In [70]:
#change entry type ids to enums
full_log['Log Entry Type'].replace({1:'SENSOR', 2:'STATE', 3:'ESTIMATOR_VAR', 4: 'CONTROLLER_OUTPUT', 5:'MOTOR', 6:'MSG'}, inplace = True)

# Sensor Log

In [71]:
#extract sensor entries
sensor_log = full_log[full_log['Log Entry Type'] == 'SENSOR'].copy()
#extract data for sensor entries
sensor_log[['Sensor Board ID', 'Sensor Type', 'Data']] = sensor_log['Data'].str.split(',', expand = True, n = 2)

#change sensor type ids to enums
sensor_log['Sensor Type'] = sensor_log['Sensor Type'].astype(int)
sensor_log['Sensor Type'].replace({1:'BARO', 2:'IMU', 3:'GPS', 4: 'BATTERY'}, inplace = True)

In [72]:
#extract individual sensor types
baro_log = sensor_log[sensor_log['Sensor Type'] == 'BARO'].copy()
imu_log = sensor_log[sensor_log['Sensor Type'] == 'IMU'].copy()
gps_log = sensor_log[sensor_log['Sensor Type'] == 'GPS'].copy()
battery_log = sensor_log[sensor_log['Sensor Type'] == 'BATTERY'].copy()

## Baro Log

In [73]:
#extract data for baro sensors
baro_log[['Pressure', 'Temperature', 'Sensor Board Timestamp']] = baro_log['Data'].str.split(',', expand = True)
baro_log.drop(columns = ['Data'], inplace = True)
#reorder columns
baro_log = baro_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type', 'Pressure', 'Temperature']]

#convert temperature to Celsius
fmt_tmp = lambda x: x / 100
baro_log['Temperature'] = baro_log['Temperature'].apply(pd.to_numeric).apply(fmt_tmp)
baro_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Pressure,Temperature
Log Entry,,,,,,,,
3,511,0.511,SENSOR,1,0,BARO,0,0.00
5,511,0.511,SENSOR,2,0,BARO,0,0.00
7,511,0.511,SENSOR,3,0,BARO,0,0.00
12,521,0.521,SENSOR,1,583,BARO,95352,29.09
14,521,0.521,SENSOR,2,583,BARO,95326,29.18


In [74]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
baro_log.to_csv(f'{split_logs_dir}/baro.csv')

## IMU Log

In [75]:
#generate Acc_x, Acc_y, Acc_z, Gyro_x, Gyro_y, Gyro_z
dim3 = lambda x : [x + '_' + s for s in ['x', 'y', 'z']]
imu_cols = dim3('Acc') + dim3('Gyro')

#extract data for imu sensors
imu_log[imu_cols + ['Sensor Board Timestamp']] = imu_log['Data'].str.split(',', expand = True)
imu_log.drop(columns = ['Data'], inplace = True)
#reorder columns
imu_log = imu_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Board Timestamp', 'Sensor Type'] + imu_cols]

#scale acceleration and rotation to float

#convert to gs
fmt_acc = lambda x: x / 1024
#convert to degrees per second
fmt_gyro = lambda x: x / 32.4

imu_log[dim3('Acc')] = imu_log[dim3('Acc')].apply(pd.to_numeric).apply(fmt_acc)
imu_log[dim3('Gyro')] = imu_log[dim3('Gyro')].apply(pd.to_numeric).apply(fmt_gyro)

imu_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Board Timestamp,Sensor Type,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z
Log Entry,,,,,,,,,,,,
4,511,0.511,SENSOR,1,0,IMU,0.0,0.0,0.0,0.0,0.0,0.0
6,511,0.511,SENSOR,2,0,IMU,0.0,0.0,0.0,0.0,0.0,0.0
9,511,0.511,SENSOR,3,0,IMU,0.0,0.0,0.0,0.0,0.0,0.0
13,521,0.521,SENSOR,1,0,IMU,0.0,0.0,0.0,0.0,0.0,0.0
15,521,0.521,SENSOR,2,0,IMU,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
imu_log.to_csv(f'{split_logs_dir}/imu.csv')

## GPS Log

In [77]:
gps_cols = ['Hour', 'Minute', 'Second', 'Latitude [deg]', 'Latitude [decimal]', 'Longitude [deg]', 'Longitude [decimal]', 'Satellite', 'Altitude', 'HDOP']
gps_log[gps_cols] = gps_log['Data'].str.split(',', expand = True)
gps_log = gps_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Type', ] + gps_cols]
gps_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Type,Hour,Minute,Second,Latitude [deg],Latitude [decimal],Longitude [deg],Longitude [decimal],Satellite,Altitude,HDOP
Log Entry,,,,,,,,,,,,,,,
0,201,0.201,SENSOR,1,GPS,0,0,0,0,0,0,0,0,0,0
1,201,0.201,SENSOR,2,GPS,0,0,0,0,0,0,0,0,0,0
2,203,0.203,SENSOR,3,GPS,0,0,0,0,0,0,0,0,0,0
8,502,0.502,SENSOR,1,GPS,0,0,0,0,0,0,0,0,0,0
10,502,0.502,SENSOR,2,GPS,0,0,0,0,0,0,0,0,0,0


In [78]:
gps_log.to_csv(f'{split_logs_dir}/gps.csv')

## Battery Log

In [79]:
battery_cols = ['Battery', 'Consumption', 'Current', 'Supply']
battery_log[battery_cols] = battery_log['Data'].str.split(',', expand = True)
battery_log = battery_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Sensor Board ID', 'Sensor Type', ] + battery_cols]
battery_log.head()

,Timestamp,Time_s,Log Entry Type,Sensor Board ID,Sensor Type,Battery,Consumption,Current,Supply
Log Entry,,,,,,,,,
453,1078,1.078,SENSOR,1,BATTERY,11401,0,81,3299
1379,2098,2.098,SENSOR,1,BATTERY,11395,0,142,3300
2324,3118,3.118,SENSOR,1,BATTERY,11394,0,135,3302
3269,4138,4.138,SENSOR,1,BATTERY,11389,0,147,3302
4215,5158,5.158,SENSOR,1,BATTERY,11394,0,153,3302


In [80]:
battery_log.to_csv(f'{split_logs_dir}/battery.csv')

# State Log

In [81]:
#IDLE = 1, AIRBRAKE_TEST, THRUSTING, COASTING, DESCENT, RECOVERY

In [82]:
#extract state change entries
state_log = full_log[full_log['Log Entry Type'] == 'STATE'].copy()
state_log.rename(columns={'Data': 'State'}, inplace = True)
state_log['State'] = state_log['State'].astype(int)
state_log['State'].replace({1:'IDLE', 2:'AIRBRAKE_TEST', 3:'THRUSTING', 4:'COASTING', 5:'DESCENT', 6:'RECOVERY'}, inplace = True)
state_log.head()

,Timestamp,Log Entry Type,State,Time_s
Log Entry,,,,
136,701,STATE,IDLE,0.701
144,711,STATE,IDLE,0.711
152,721,STATE,IDLE,0.721
160,731,STATE,IDLE,0.731
168,741,STATE,IDLE,0.741


In [83]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
state_log.to_csv(f'{split_logs_dir}/state_transitions.csv')

# Estimator Variable Log

In [84]:
#extract estimator var entries
estimator_var_log = full_log[full_log['Log Entry Type'] == 'ESTIMATOR_VAR'].copy()
estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']] = estimator_var_log['Data'].str.split(',', expand = True)

estimator_var_log.drop(columns = ['Data'], inplace = True)
#reorder columns
estimator_var_log = estimator_var_log[['Timestamp', 'Time_s', 'Log Entry Type', 'World Position (z)', 'Rocket Velocity (z)', 'Rocket Acceleration (z)']]

fmt_est = lambda x: x / 1000
estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']] = estimator_var_log[['World Position (z)', 'Rocket Velocity (z)',  'Rocket Acceleration (z)']].apply(pd.to_numeric).apply(fmt_est)
estimator_var_log.head()

,Timestamp,Time_s,Log Entry Type,World Position (z),Rocket Velocity (z),Rocket Acceleration (z)
Log Entry,,,,,,
66,601,0.601,ESTIMATOR_VAR,0.0,0.0,0.0
73,611,0.611,ESTIMATOR_VAR,0.0,0.0,0.0
80,621,0.621,ESTIMATOR_VAR,0.0,0.0,0.0
87,631,0.631,ESTIMATOR_VAR,0.0,0.0,0.0
94,641,0.641,ESTIMATOR_VAR,0.0,0.0,0.0


In [85]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
estimator_var_log.to_csv(f'{split_logs_dir}/estimator_var.csv')

# Controller Output Log

In [86]:
#extract estimator var entries
controller_out_log = full_log[full_log['Log Entry Type'] == 'CONTROLLER_OUTPUT'].copy()
controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']] = controller_out_log['Data'].str.split(',', expand = True)

controller_out_log.drop(columns = ['Data'], inplace = True)
#reorder columns
controller_out_log = controller_out_log[['Timestamp', 'Time_s', 'Log Entry Type', 'Controller Output', 'Reference Error', 'Integrated Error']]

fmt_cnt = lambda x: x / 1000
controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']] = controller_out_log[['Controller Output', 'Reference Error', 'Integrated Error']].apply(pd.to_numeric).apply(fmt_cnt)
controller_out_log.head()

,Timestamp,Time_s,Log Entry Type,Controller Output,Reference Error,Integrated Error
Log Entry,,,,,,
216,801,0.801,CONTROLLER_OUTPUT,0.0,0.0,0.0
225,811,0.811,CONTROLLER_OUTPUT,0.0,0.0,0.0
234,821,0.821,CONTROLLER_OUTPUT,0.0,0.0,0.0
243,831,0.831,CONTROLLER_OUTPUT,0.0,0.0,0.0
252,841,0.841,CONTROLLER_OUTPUT,0.0,0.0,0.0


In [87]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
controller_out_log.to_csv(f'{split_logs_dir}/controller_out.csv')

# Message Log

In [88]:
#extract message entries
msg_log = full_log[full_log['Log Entry Type'] == 'MOTOR'].copy()
msg_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,
1369,2069,MOTOR,"-2,0",2.069
1416,2120,MOTOR,"-2,0",2.120
1462,2168,MOTOR,"-2,0",2.168
1515,2223,MOTOR,"-2,0",2.223
1554,2268,MOTOR,"-2,0",2.268


In [89]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
msg_log.to_csv(f'{split_logs_dir}/motor.csv')

In [90]:
#extract message entries
msg_log = full_log[full_log['Log Entry Type'] == 'MSG'].copy()
msg_log.head()

,Timestamp,Log Entry Type,Data,Time_s
Log Entry,,,,


In [91]:
#save to csv; read as pd.read_csv(loc, index_col = 0)
msg_log.to_csv(f'{split_logs_dir}/messages.csv')